In [ ]:
!pip install datasets
!pip install accelerate
!pip install transformers[torch]
!pip install torch

import pandas as pd
import spacy
import nltk
import numpy as np
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DistilBertConfig, DistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from nltk.sentiment.vader import SentimentIntensityAnalyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:

nlp_spacy = spacy.load('en_core_web_sm')


nltk.download('vader_lexicon')
nltk.download('punkt')
sia = SentimentIntensityAnalyzer()


model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


config = DistilBertConfig.from_pretrained(model_name, num_labels=3)
model = DistilBertForSequenceClassification.from_pretrained(model_name, config=config)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:

def tokenize(text):
    return nltk.word_tokenize(text)


def pos_tagging(text):
    doc = nlp_spacy(text)
    return [(token.text, token.pos_) for token in doc]


def sentiment_analysis(text):
    return sia.polarity_scores(text)


def preprocess_data(texts, labels):
    tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    dataset = Dataset.from_dict({
        'input_ids': tokenized_texts['input_ids'],
        'attention_mask': tokenized_texts['attention_mask'],
        'labels': labels
    })
    return dataset


def prepare_datasets(df):
    train_texts, test_texts, train_labels, test_labels = train_test_split(df['tweet'], df['class'], test_size=0.2, stratify=df['class'])
    train_labels = torch.tensor(train_labels.values)
    test_labels = torch.tensor(test_labels.values)
    train_dataset = preprocess_data(train_texts.tolist(), train_labels)
    test_dataset = preprocess_data(test_texts.tolist(), test_labels)
    return DatasetDict({'train': train_dataset, 'test': test_dataset})


In [ ]:

df = pd.read_csv('/content/labeled_data.csv')


print(df['class'].value_counts())


datasets = prepare_datasets(df)


class_counts = df['class'].value_counts().to_dict()
total_samples = len(df)
class_weights = {k: total_samples/v for k, v in class_counts.items()}
class_weights_tensor = torch.tensor([class_weights[i] for i in range(3)], dtype=torch.float)


training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)


def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    accuracy = accuracy_score(p.label_ids, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(device)
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor.to(logits.device))
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss


trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=datasets['train'],
    eval_dataset=datasets['test'],
    compute_metrics=compute_metrics,
)


trainer.train()
results = trainer.evaluate()


print(f"Evaluation Results: {results}")

class
1    19190
2     4163
0     1430
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.441800,0.494145,0.865241,0.911259,0.865241,0.881904
2,0.303800,0.526593,0.905386,0.914341,0.905386,0.909246
3,0.339900,0.559049,0.900343,0.913356,0.900343,0.905855
4,0.697200,0.753729,0.906798,0.908834,0.906798,0.907773
5,0.230200,0.851916,0.904983,0.906065,0.904983,0.905513


Evaluation Results: {'eval_loss': 0.8519161343574524, 'eval_accuracy': 0.9049828525317732, 'eval_precision': 0.9060647827709943, 'eval_recall': 0.9049828525317732, 'eval_f1': 0.905512917665694, 'eval_runtime': 66.1008, 'eval_samples_per_second': 74.992, 'eval_steps_per_second': 4.69, 'epoch': 5.0}


In [ ]:

def analyze_text_for_threats(text):
    tokens = tokenize(text)
    pos_tags = pos_tagging(text)
    sentiment = sentiment_analysis(text)
    inputs = tokenizer(text, return_tensors='pt').to(device)
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=1)
    threats = {"label": "threat" if probs[0][1] > 0.5 else "non-threat", "score": probs[0][1].item()}
    return {
        "text": text,
        "tokens": tokens,
        "pos_tags": pos_tags,
        "sentiment": sentiment,
        "threats": threats
    }


if __name__ == "__main__":
    sample_text = "I will find you and make you pay for this."
    analysis = analyze_text_for_threats(sample_text)
    print(f"Text: {analysis['text']}")
    print(f"Tokens: {analysis['tokens']}")
    print(f"POS Tags: {analysis['pos_tags']}")
    print(f"Sentiment: {analysis['sentiment']}")
    print(f"Threat Detection: {analysis['threats']}")
    print(f"Evaluation Results: {results}")

Text: I will find you and make you pay for this.
Tokens: ['I', 'will', 'find', 'you', 'and', 'make', 'you', 'pay', 'for', 'this', '.']
POS Tags: [('I', 'PRON'), ('will', 'AUX'), ('find', 'VERB'), ('you', 'PRON'), ('and', 'CCONJ'), ('make', 'VERB'), ('you', 'PRON'), ('pay', 'VERB'), ('for', 'ADP'), ('this', 'PRON'), ('.', 'PUNCT')]
Sentiment: {'neg': 0.149, 'neu': 0.851, 'pos': 0.0, 'compound': -0.1027}
Threat Detection: {'label': 'non-threat', 'score': 0.003478245809674263}
Evaluation Results: {'eval_loss': 0.8519161343574524, 'eval_accuracy': 0.9049828525317732, 'eval_precision': 0.9060647827709943, 'eval_recall': 0.9049828525317732, 'eval_f1': 0.905512917665694, 'eval_runtime': 66.1008, 'eval_samples_per_second': 74.992, 'eval_steps_per_second': 4.69, 'epoch': 5.0}
